In [1]:
!pip install pandas faiss-cpu sentence-transformers langchain openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [4]:
import pandas as pd

# Use the correct filename with space, wrapped in quotes
df = pd.read_csv("Training Dataset.csv")
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
def convert_row_to_text(row):
    return f"""Applicant {row['Loan_ID']} is a {row['Gender']} {row['Married']} applicant with {row['Education']} background.
    Self_Employed: {row['Self_Employed']}. ApplicantIncome: {row['ApplicantIncome']},
    LoanAmount: {row['LoanAmount']}, Credit_History: {row['Credit_History']}.
    Loan_Status: {'Approved' if row['Loan_Status'] == 'Y' else 'Rejected'}."""

# Convert all rows into natural language chunks
documents = df.apply(convert_row_to_text, axis=1).tolist()

# View a sample
documents[:3]



['Applicant LP001002 is a Male No applicant with Graduate background. \n    Self_Employed: No. ApplicantIncome: 5849, \n    LoanAmount: nan, Credit_History: 1.0. \n    Loan_Status: Approved.',
 'Applicant LP001003 is a Male Yes applicant with Graduate background. \n    Self_Employed: No. ApplicantIncome: 4583, \n    LoanAmount: 128.0, Credit_History: 1.0. \n    Loan_Status: Rejected.',
 'Applicant LP001005 is a Male Yes applicant with Graduate background. \n    Self_Employed: Yes. ApplicantIncome: 3000, \n    LoanAmount: 66.0, Credit_History: 1.0. \n    Loan_Status: Approved.']

In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load lightweight sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for our text chunks
doc_embeddings = model.encode(documents, convert_to_numpy=True)

# Build FAISS index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
def retrieve_docs(query, k=5):
    query_vec = model.encode([query])
    distances, indices = index.search(query_vec, k)
    return [documents[i] for i in indices[0]]


In [17]:
!pip install transformers accelerate


In [18]:
from transformers import pipeline

# Load FLAN-T5 model (free, no key needed)
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [19]:
def generate_answer_with_hf(query, context):
    prompt = f"Answer the question based on the context.\nContext: {context}\nQuestion: {query}"
    response = qa_pipeline(prompt, max_new_tokens=100, do_sample=True)
    return response[0]['generated_text']


In [20]:
def rag_chatbot(query):
    # Retrieve top relevant chunks
    relevant_docs = retrieve_docs(query)

    # Join them as a single context
    context = "\n".join(relevant_docs)

    # Generate answer using Hugging Face
    answer = generate_answer_with_hf(query, context)

    return answer


In [21]:
response = rag_chatbot("What kind of applicants usually get loan approval?")
print("🤖 Answer:", response)


🤖 Answer: Male Yes
